Este notebook construye una muestra representativa de la población lectora con base en variables clave del dataset `books_rating`

A continuación se describen las variables seleccionadas como representativas del comportamiento de la población:

| Variable            | Dominio                          | Estadísticas conocidas                           | Comentarios adicionales |
|---------------------|-----------------------------|--------------------------------|---------------------------------------------|
| `review/score`      | [1.0 – 5.0]                 | Moda: 5.0, Media aprox: 4.2    | Escala de satisfaccion                      |
| `review/helpfulness`| [0/0 – 10/10]               | Rango típico 1–10              | Requiere limpieza y transformación a entero |
| `user_id`           | IDs únicos por usuario      | Frecuencia variable            | Clasifica por volumen de participación      |


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, split, count
from pyspark.sql.types import FloatType, IntegerType

spark = SparkSession.builder.appName('MuestreoBooksRating').getOrCreate()

# Cargar dataset
df = spark.read.option('header', 'true').csv('Books_rating.csv')
df.printSchema()
df.select('review/score', 'review/helpfulness', 'user_id').show(5) ##Ejemplo de variables de participacion

25/05/03 21:55:55 WARN Utils: Your hostname, MacBook-Pro-de-Juan.local resolves to a loopback address: 127.0.0.1; using 192.168.100.16 instead (on interface en0)
25/05/03 21:55:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/03 21:55:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


root
 |-- Id: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- User_id: string (nullable = true)
 |-- profileName: string (nullable = true)
 |-- review/helpfulness: string (nullable = true)
 |-- review/score: string (nullable = true)
 |-- review/time: string (nullable = true)
 |-- review/summary: string (nullable = true)
 |-- review/text: string (nullable = true)

+------------+------------------+--------------+
|review/score|review/helpfulness|       user_id|
+------------+------------------+--------------+
|         4.0|               7/7| AVCGYZL8FQQTD|
|         5.0|             10/10|A30TK6U7DNS82R|
|         5.0|             10/11|A3UH4UZ4RSVO82|
|         4.0|               7/7|A2MVUWT453QH61|
|         4.0|               3/3|A22X4XUPKF66MR|
+------------+------------------+--------------+
only showing top 5 rows



25/05/03 21:56:06 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [2]:
# Remover advertencias
spark.sparkContext.setLogLevel("ERROR")

In [3]:
# Limpieza de datos 
df_clean = df.filter((col("review/score").isNotNull()) & 
                     (col("review/helpfulness").isNotNull()) & 
                     (col("user_id").isNotNull()))

In [4]:
df = df_clean

In [5]:
# Convertir columnas relevantes a int
df = df.withColumn("Score_num", col("review/score").cast(FloatType()))
df = df.withColumn("Helpfulness_num", split(col("review/helpfulness"), "/")[0].cast(IntegerType()))

# Clasificar score
df = df.withColumn("score_group", when(col("Score_num") >= 4, "Alta").otherwise("Baja"))
# Clasificar helpfulness
df = df.withColumn("helpfulness_group", when(col("Helpfulness_num") >= 8, "Alta").otherwise("Baja"))

In [6]:
# Remover valores no convertidos correctamente
df = df.filter((col("Score_num").isNotNull()) & (col("Helpfulness_num").isNotNull()))

In [7]:
# Calcular cantidad de reseñas por usuario
user_reviews = df.groupBy("user_id").agg(count("user_id").alias("review_count"))
df = df.join(user_reviews, on="user_id", how="left")


In [8]:
# Clasificación más detallada de usuarios según frecuencia de participación
df = df.withColumn(
    "user_group_detailed",
    when(col("review_count") >= 20, "20+ reseñas")
    .when((col("review_count") >= 10) & (col("review_count") < 20), "10-19 reseñas")
    .when((col("review_count") >= 5) & (col("review_count") < 10), "5-9 reseñas")
    .when((col("review_count") >= 2) & (col("review_count") < 5), "2-4 reseñas")
    .otherwise("1 reseña")
)

In [9]:
# Mostrar distribución con mejor formato
from pyspark.sql.functions import format_number
distribution = df.groupBy("user_group_detailed").count().orderBy("count", ascending=False)
distribution = distribution.withColumn("count", format_number(col("count"), 0))
distribution.show(truncate=False, n=100)

+-------------------+-------+
|user_group_detailed|count  |
+-------------------+-------+
|1 reseña           |690,877|
|2-4 reseñas        |580,802|
|20+ reseñas        |540,951|
|5-9 reseñas        |338,868|
|10-19 reseñas      |268,931|
+-------------------+-------+



In [10]:
# Revisión de distribución conjunta de variables de caracterización
df.groupBy("score_group", "helpfulness_group", "user_group_detailed").count().orderBy("count", ascending=False).show()

[Stage 15:>                                                       (0 + 12) / 13]

+-----------+-----------------+-------------------+------+
|score_group|helpfulness_group|user_group_detailed| count|
+-----------+-----------------+-------------------+------+
|       Alta|             Baja|           1 reseña|482065|
|       Alta|             Baja|        2-4 reseñas|402632|
|       Alta|             Baja|        20+ reseñas|356614|
|       Alta|             Baja|        5-9 reseñas|243589|
|       Alta|             Baja|      10-19 reseñas|190730|
|       Baja|             Baja|           1 reseña|104137|
|       Baja|             Baja|        2-4 reseñas| 97797|
|       Alta|             Alta|        20+ reseñas| 85625|
|       Baja|             Baja|        20+ reseñas| 78467|
|       Alta|             Alta|           1 reseña| 72828|
|       Baja|             Baja|        5-9 reseñas| 53827|
|       Alta|             Alta|        2-4 reseñas| 53819|
|       Baja|             Baja|      10-19 reseñas| 44771|
|       Baja|             Alta|           1 reseña| 3184

In [11]:
# Cálculo de ocurrencias por combinación
comb_counts = df.groupBy("score_group", "helpfulness_group", "user_group_detailed").count()
total = df.count()
comb_probs = comb_counts.withColumn("probabilidad", (col("count") / total))
comb_probs.orderBy(col("probabilidad").desc()).show(truncate=False)

[Stage 27:>                                                       (0 + 12) / 13]

+-----------+-----------------+-------------------+------+--------------------+
|score_group|helpfulness_group|user_group_detailed|count |probabilidad        |
+-----------+-----------------+-------------------+------+--------------------+
|Alta       |Baja             |1 reseña           |482065|0.1991651066815015  |
|Alta       |Baja             |2-4 reseñas        |402632|0.1663473706520621  |
|Alta       |Baja             |20+ reseñas        |356614|0.14733503854068844 |
|Alta       |Baja             |5-9 reseñas        |243589|0.1006387710608326  |
|Alta       |Baja             |10-19 reseñas      |190730|0.07880008048160057 |
|Baja       |Baja             |1 reseña           |104137|0.04302419116611146 |
|Baja       |Baja             |2-4 reseñas        |97797 |0.0404048207982965  |
|Alta       |Alta             |20+ reseñas        |85625 |0.035375960212012   |
|Baja       |Baja             |20+ reseñas        |78467 |0.03241863322576287 |
|Alta       |Alta             |1 reseña 

In [12]:
# Muestreo por todas las combinaciones posibles (2x2x7 = 28 combinaciones)
from itertools import product
selected_columns = ["user_id", "review/helpfulness", "review/score", "review/summary", "Score_num", "user_group_detailed"]

for s, h in product(["Alta", "Baja"], repeat=2):
    for u in ["20+ reseñas", "10-19 reseñas", "5-9 reseñas", "2-4 reseñas", "1 reseña"]:
        subset = df.filter((col("score_group") == s) &
                           (col("helpfulness_group") == h) &
                           (col("user_group_detailed") == u))
        count_subset = subset.count()
        print(f"Muestra: Score={s}, Helpfulness={h}, Usuario={u}, Registros={count_subset}")
        muestra = subset.select(*selected_columns).sample(False, 0.1, seed=42)
        muestra.show(5, truncate=False)

Muestra: Score=Alta, Helpfulness=Alta, Usuario=20+ reseñas, Registros=85625


+--------------+------------------+------------+------------------------------------------+---------+-------------------+
|user_id       |review/helpfulness|review/score|review/summary                            |Score_num|user_group_detailed|
+--------------+------------------+------------+------------------------------------------+---------+-------------------+
|A1075MZNVRMSEO|17/19             |4.0         |A great book about destructive temptations|4.0      |20+ reseñas        |
|A1075MZNVRMSEO|24/24             |5.0         |A great mid-size dictionary               |5.0      |20+ reseñas        |
|A1075MZNVRMSEO|10/10             |5.0         |A wonderful textbook for serious students.|5.0      |20+ reseñas        |
|A11L4SBY7NCSZU|35/36             |5.0         |Fantastic                                 |5.0      |20+ reseñas        |
|A11LNPG39A2ZV4|28/30             |4.0         |Best for Beginners                        |4.0      |20+ reseñas        |
+--------------+--------

Muestra: Score=Alta, Helpfulness=Alta, Usuario=10-19 reseñas, Registros=23537


+--------------+------------------+------------+---------------------------------------------+---------+-------------------+
|user_id       |review/helpfulness|review/score|review/summary                               |Score_num|user_group_detailed|
+--------------+------------------+------------+---------------------------------------------+---------+-------------------+
|A109DZXQULGEUK|22/24             |5.0         |A modern classic                             |5.0      |10-19 reseñas      |
|A10FBJXMQPI0LL|10/13             |4.0         |I really enjoyed this book!                  |4.0      |10-19 reseñas      |
|A10MR5DEPRCX98|10/16             |5.0         |FORGET HAMLET, A NEW GUY IS IN TOWN          |5.0      |10-19 reseñas      |
|A142SWWCTIKL0H|42/42             |5.0         |Hard to believe this was written decades ago!|5.0      |10-19 reseñas      |
|A14YHC72SHZHRT|27/27             |4.0         |Delicious food; a couple of minor gripes     |4.0      |10-19 reseñas      |


Muestra: Score=Alta, Helpfulness=Alta, Usuario=5-9 reseñas, Registros=27862


...

Muestra: Score=Baja, Helpfulness=Baja, Usuario=2-4 reseñas, Registros=97797


+--------------+------------------+------------+-------------------------------------------------------+---------+-------------------+
|user_id       |review/helpfulness|review/score|review/summary                                         |Score_num|user_group_detailed|
+--------------+------------------+------------+-------------------------------------------------------+---------+-------------------+
|A104EP737ZROHQ|4/10              |1.0         |UGH!!                                                  |1.0      |2-4 reseñas        |
|A105XBOOCZNISQ|7/19              |1.0         |Self published, unreadable, apologia for a child rapist|1.0      |2-4 reseñas        |
|A1086X5YV9M4QF|0/0               |2.0         |Coffin to Heaven                                       |2.0      |2-4 reseñas        |
|A10SJ1NNSQ1CB |3/16              |1.0         |Too much bias to be objective.                         |1.0      |2-4 reseñas        |
|A10W5NFZ9PLX4K|3/10              |1.0         |A pure,

Muestra: Score=Baja, Helpfulness=Baja, Usuario=1 reseña, Registros=104137


[Stage 308:====================================================>  (21 + 1) / 22]

+--------------+------------------+------------+------------------------------------------------------------+---------+-------------------+
|user_id       |review/helpfulness|review/score|review/summary                                              |Score_num|user_group_detailed|
+--------------+------------------+------------+------------------------------------------------------------+---------+-------------------+
|A102LSCF3BSKUR|0/8               |1.0         |New Spring by Robert Jordan                                 |1.0      |1 reseña           |
|A107ZPHSRVHZ00|1/1               |2.0         |Kindle Download/Publisher Proofreading Errors               |2.0      |1 reseña           |
|A109PJ5IB5R6N2|4/6               |3.0         |Out of date content! However, good explanation, good CD-ROM.|3.0      |1 reseña           |
|A10R2Z2FQLPYEY|1/2               |1.0         |kindle version full of typos                                |1.0      |1 reseña           |
|A10Y0YM81YDLIT|1/1 

**Técnica aplicada**: *Muestreo estratificado* sobre combinaciones de score, utilidad y tipo de usuario.

Se utiliza esta técnica porque permite garantizar que cada subgrupo esté representado proporcionalmente en la muestra.

**Justificación:**
- Los usuarios muy activos pueden dominar el conjunto si no se estratifica.
- Las reseñas útiles y bien puntuadas tienen mayor influencia sobre recomendaciones.
- El análisis cruzado de estas variables representa diferentes comportamientos del lector.